In [43]:
from arcgis.gis import GIS

gis = GIS("https://www.arcgis.com/home/", "gregbrunner_slugis")

Enter password: ········


In [2]:
data = "prcl_shape.zip"
stl = gis.content.add({"title":"St. Louis parcel data from STL's Open Data site", "tags":"St. Louis, parcels"}, data)

In [3]:
published_stl = stl.publish()

In [4]:
published_stl

<Item title:"St. Louis parcel data from STL's Open Data site" type:Feature Layer Collection owner:gregbrunner_slugis>

In [5]:
stl_map = gis.map('St. Louis, MO')
stl_map

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

In [6]:
stl_map.add_layer(published_stl)

## Ignore this :)

In [ ]:
high_estimate = '../data-2017/parcel/high-estimate/parcel-data-2017-highEstimate.csv'
h_est = gis.content.add({"title":"high_estimate", "tags":"St. Louis, parcels, vacancy"}, high_estimate)

from arcgis.features import analysis

enriched_parcels = analysis.join_features(target_layer=published_stl, join_layer=h_est, attribute_relationship='HANDLE', gis=gis)

## This didn't work. Let's just join dataframes...

In [45]:
#from arcgis.features import FeatureLayer

#url = 'https://services1.arcgis.com/g2TonOxuRkIqSOFx/arcgis/rest/services/prcl_shape/FeatureServer/0'

#slugis = GIS("https://www.arcgis.com/home/", "gregbrunner_slugis")

#parcel_fl = FeatureLayer(url, gis = gis)

In [17]:
from arcgis.features import FeatureLayer

parcel_fl = FeatureLayer(url = published_stl.url, gis = gis)

In [47]:
parcels_sdf = published_stl.layers[0].query().df
#parcels_sdf = parcel_fl.query().df

In [48]:
len(parcels_sdf)

127758

In [49]:
parcels_sdf.HANDLE.dtype

dtype('O')

In [50]:
parcels_sdf['H'] = parcels_sdf['HANDLE'].astype(float)

In [51]:
parcels_sdf.H.dtype

dtype('float64')

In [52]:
import pandas as pd

In [53]:
low_estimate = '../data-2017/parcel/low-estimate/parcel-data-2017-lowEstimate.csv'
high_estimate = '../data-2017/parcel/high-estimate/parcel-data-2017-highEstimate.csv'

In [54]:
estimate = high_estimate

In [55]:
h_est_df = pd.read_csv(estimate)

In [56]:
h_est_df = h_est_df.rename(columns={ 'handle' : "H"})

In [57]:
h_est_df.H.dtype

dtype('float64')

In [58]:
len(h_est_df)

48013

In [59]:
h_est_df.head()

,H,vacantBinary,cityStatus
0,1.000100e+10,True,False
1,1.000100e+10,True,False
2,1.000100e+10,True,False
3,1.001400e+10,True,False
4,1.001400e+10,True,False


In [60]:
from arcgis.features import SpatialDataFrame as spd

In [61]:
merged_data = spd.merge(parcels_sdf, h_est_df, how = 'left', left_on='H', right_on='H')

In [62]:
merged_data.head()

,FID,HANDLE,Shape__Area,Shape__Length,SHAPE,H,vacantBinary,cityStatus
0,1,10329000045,1830.843750,179.595654,"{'rings': [[[-10039885.1163153, 4671794.922912...",1.032900e+10,NaN,NaN
1,2,10001000005,1020.195312,130.846919,"{'rings': [[[-10039556.032893, 4667225.8128700...",1.000100e+10,True,False
2,3,10329000075,1830.839844,179.595194,"{'rings': [[[-10039873.7565212, 4671765.761943...",1.032900e+10,NaN,NaN
3,4,10001000010,2003.324219,273.216695,"{'rings': [[[-10039533.5723971, 4667298.017724...",1.000100e+10,True,False
4,5,10329000105,1830.824219,179.594684,"{'rings': [[[-10039862.3967913, 4671736.601044...",1.032900e+10,NaN,NaN


In [35]:
len(merged_data[merged_data['vacantBinary']==True])

31473

In [36]:
empty_parcels = merged_data[merged_data['vacantBinary']==True].copy()

In [37]:
empty_parcels.reset_index(inplace=True, drop=True)
empty_parcels.head()

,FID,HANDLE,Shape__Area,Shape__Length,SHAPE,H,vacantBinary,cityStatus
0,2,10001000005,1020.195312,130.846919,"{'rings': [[[-10039556.032893, 4667225.8128700...",1.000100e+10,True,False
1,4,10001000010,2003.324219,273.216695,"{'rings': [[[-10039533.5723971, 4667298.017724...",1.000100e+10,True,False
2,7,10330000030,244.117188,78.191878,"{'rings': [[[-10039983.5732021, 4671883.351409...",1.033000e+10,True,False
3,8,10001000032,323.976562,88.823036,"{'rings': [[[-10039503.905549, 4667209.6188938...",1.000100e+10,True,False
4,14,10014000005,1417.808594,176.194846,"{'rings': [[[-10039171.9315976, 4668734.117673...",1.001400e+10,True,False


In [63]:
merged_data.to_featureclass(out_location=r'C:\Users\greg6750\Documents\IPython Notebooks\vacancy-data\data-exploration\vacancy.gdb', 
                              out_name="high_estimate_all")

'C:\\Users\\greg6750\\Documents\\IPython Notebooks\\vacancy-data\\data-exploration\\vacancy.gdb\\high_estimate_all'

In [65]:
#merged_data[merged_data['vacantBinary']==True].to_featureclass(out_location=r'C:\Users\greg6750\Documents\IPython Notebooks\vacancy-data\data-exploration\vacancy.gdb', 
#                              out_name="vacant_low_estimate")

In [64]:
fs = gis.content.import_data(merged_data, title='STL Vacancy - High Estimate')

In [ ]:
## Extra

In [ ]:
#sdf = sdf[sdf.SHAPE.notna()].copy()